## One basic Mnist TF Model

In [1]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import tensorflow as tf

/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

In [2]:
mnist = read_data_sets("./dataset", one_hot=True, reshape=False, validation_size=0)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./dataset/t10k-images-idx3-ubyte.gz
Extracting ./dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print (tf.__version__)

1.14.0


In [4]:
CKPT_DIR = 'new_ckpt_dir'

In [5]:
class Network(object):
    def __init__(self):
        self.learning_rate = 0.001
        self.global_step = tf.Variable(0,trainable = False)
        self.x = tf.placeholder(tf.float32, [None, 28, 28, 1], name = "x_input")
        self.label = tf.placeholder(tf.float32,[None,10], name = 'y_input')
        self.w = tf.Variable(tf.zeros([784,10]))
        self.b = tf.Variable(tf.zeros([10]))
        self.XX = tf.reshape(self.x, shape=[-1, 784])
        self.y = tf.nn.softmax(tf.matmul(self.XX,self.w) + self.b, name = 'z_output')
        self.loss = -tf.reduce_mean(self.label * tf.log(self.y) + 1e-10)
        self.train = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss,global_step = self.global_step)
        predict = tf.equal(tf.argmax(self.label,1),tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(predict,tf.float32))

In [6]:
class Train(object):
    
    def __init__(self):
        self.net = Network()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def train(self):
        batch_size = 64
        train_step = 10000
        step = 0
        saver = tf.train.Saver(max_to_keep = 0)
        ckpt = tf.train.get_checkpoint_state(CKPT_DIR)
        if ckpt and ckpt.get_checkpoint_state(CKPT_DIR):
            saver.restore(self.sess,ckpt.model_checkpoint_path)
            step = self.sess.run(self.net.global_step) # Fetch the global step from the CKPT model
            print('continue from')
            print('  -> Minibatch update : ',step)
        while step < train_step:
            x, label = mnist.train.next_batch(batch_size)
            _,loss = self.sess.run([self.net.train,self.net.loss], feed_dict = {self.net.x: x,self.net.label:label})
            step = self.sess.run(self.net.global_step)
            if step % 1000 == 0:
                print('No %6d step, current loss: %.3f'%(step,loss))
            if step % 10000 == 0:
                saver.save(self.sess,CKPT_DIR + '/model',global_step = step)
    
    def calculate_accuracy(self):
        test_x = mnist.test.images
        test_label = mnist.test.labels
        acc = self.sess.run(self.net.accuracy,feed_dict = {self.net.x:test_x,self.net.label:test_label})
        print("acc: %.3f, Number of pic test: %d " % (acc, len(test_label)))

In [7]:
if __name__ == '__main__':
    model = Train()
    model.train()
    model.calculate_accuracy()

No   1000 step, current loss: 0.219
No   2000 step, current loss: 0.209
No   3000 step, current loss: 0.202
No   4000 step, current loss: 0.194
No   5000 step, current loss: 0.183
No   6000 step, current loss: 0.183
No   7000 step, current loss: 0.170
No   8000 step, current loss: 0.170
No   9000 step, current loss: 0.154
No  10000 step, current loss: 0.157
INFO:tensorflow:new_ckpt_dir/model-10000 is not in all_model_checkpoint_paths. Manually adding it.
acc: 0.791, Number of pic test: 10000 


In [8]:
tf.reset_default_graph()

In [26]:
import numpy as np
x, label = mnist.train.next_batch(32)

In [57]:
y, new_label = mnist.train.next_batch(32)

In [58]:
print (y.shape)

(32, 28, 28, 1)


In [29]:
print (x.shape)

(32, 28, 28, 1)


In [59]:
def model_restore_inference():
    sess = tf.Session()
    saver = tf.train.import_meta_graph('./new_ckpt_dir/model-10000.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./new_ckpt_dir'))
    input_x = sess.graph.get_tensor_by_name('x_input:0')
    logits = sess.graph.get_tensor_by_name('z_output:0')
    ret = sess.run(logits, {input_x:y})
    print (ret)
    print(np.argmax(ret, axis=1))

In [60]:
model_restore_inference()

INFO:tensorflow:Restoring parameters from ./new_ckpt_dir/model-10000
[[0.10841706 0.04617622 0.05912366 0.11703159 0.08475072 0.15665607
  0.04944148 0.09974264 0.1467772  0.13188341]
 [0.22704913 0.03240506 0.09899905 0.22519769 0.05501195 0.11319795
  0.07596913 0.03990334 0.08369131 0.04857536]
 [0.09220583 0.1146492  0.11611874 0.10832658 0.08583894 0.10964114
  0.09676941 0.08767551 0.11509439 0.07368019]
 [0.05658335 0.05745472 0.08811751 0.07307006 0.18997613 0.08323205
  0.11314528 0.09622805 0.11054733 0.13164555]
 [0.04586288 0.16799837 0.09570821 0.08433018 0.10872564 0.07173033
  0.08054798 0.12296626 0.11433949 0.10779069]
 [0.21212715 0.03820959 0.10356621 0.08255032 0.06378555 0.06918816
  0.2260393  0.05961361 0.08509041 0.05982979]
 [0.04118611 0.3221302  0.10235254 0.08764888 0.05758281 0.06247237
  0.07284125 0.07626836 0.10535688 0.07216051]
 [0.14679338 0.02333928 0.08228805 0.040509   0.1910505  0.05809385
  0.11407567 0.11011762 0.07561462 0.15811794]
 [0.2481625

In [61]:
from collections import OrderedDict
data = OrderedDict()

In [62]:
unit = {}
unit['first'] = y.tolist()

In [63]:
import json
with open ("data1.json", "w") as fp:
    fp.write(json.dumps(unit, indent=4))

OrderedDict()


## How to reload model from the saved CPKT files

### Use tensorboard to visualize the model

In [7]:
Net_structure = Network()

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
wirter = tf.summary.FileWriter('logs/',sess.graph)

#### $ tensorboard --logdir='/home/linbin/Tensorflow_Inference_C/logs'

### Then we could use tensorboard to see the net structure from our web broswer

## How to reload CKPT Model from local and do inference

In [15]:
def get_model_filenames(model_dir, model_exp):
    files = os.listdir(model_dir)
    meta_files = [s for s in files if s.endswith('.meta')]
    if len(meta_files)==0:
        raise load_modelValueError('No meta file found in the model directory (%s)' % model_dir)
    #fetch the goal meta file
    goal_meta_file = meta_files[0]
    for each_file in meta_files:
        if each_file.startswith(model_exp):
            goal_meta_file = each_file
            break
    ckpt = tf.train.get_checkpoint_state(model_dir)     # 通过checkpoint文件找到模型文件名
    print (ckpt.model_checkpoint_path)
    if ckpt and ckpt.model_checkpoint_path:
        # ckpt.model_checkpoint_path表示模型存储的位置，不需要提供模型的名字，它回去查看checkpoint文件
        ckpt_file = os.path.basename(ckpt.model_checkpoint_path)
        return goal_meta_file, ckpt_file
    
    meta_files = [s for s in files if '.ckpt' in s]
    max_step = -1
    for f in files:
        step_str = re.match(r'(^model-[\w\- ]+.ckpt-(\d+))', f)
        if step_str is not None and len(step_str.groups())>=2:
            step = int(step_str.groups()[1])
            if step > max_step:
                max_step = step
                ckpt_file = step_str.groups()[0]
    return goal_meta_file, ckpt_file

In [16]:
meta_file, ckpt_file = get_model_filenames(model_dir, model_exp)

./new_ckpt_dir/model-10000


In [17]:
print ('Metagraph file: %s' % meta_file)
print ('Checkpoint file: %s' % ckpt_file)

Metagraph file: model-10000.meta
Checkpoint file: model-10000


In [18]:
from tensorflow.python import pywrap_tensorflow

In [19]:
reader = pywrap_tensorflow.NewCheckpointReader(os.path.join(model_dir, ckpt_file))
var_to_shape_map = reader.get_variable_to_shape_map()
for key in var_to_shape_map:
    print("tensor_name: ", key)

('tensor_name: ', 'Variable')
('tensor_name: ', 'Variable_2')
('tensor_name: ', 'Variable_1')


### Get the tensor W in this model

In [20]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(os.path.join(model_dir, meta_file))
    saver.restore(tf.get_default_session(), os.path.join(model_dir, ckpt_file))
    print (type(sess.run(tf.get_default_graph().get_tensor_by_name("Variable_1:0"))))

INFO:tensorflow:Restoring parameters from ./new_ckpt_dir/model-10000
<type 'numpy.ndarray'>
